In [5]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [44]:
import os
import json
import time
import pandas as pd
import requests

from langchain import hub
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import logging
from openai import OpenAI
import openai
import traceback


In [7]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'EXP04_GC' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: EXP04_GC


In [8]:
load_dotenv()

True

In [9]:
# 데이터 구성
file_path = '../data/documents.jsonl'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(page_content=data['content'], metadata={
        'docid': data['docid'],
        'src': data['src'],
        'source': '/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/documents.jsonl',
        'seq_num': seq_num,
    })
    documents.append(doc)
    seq_num += 1


In [10]:
# Splitter
splitter = CharacterTextSplitter(
    separator='',
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
split_documents = splitter.split_documents(documents)

In [11]:
# Embedding
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY, 
    model="solar-embedding-1-large"
)

In [12]:
# 벡터 저장소 생성
# pip install faiss-cpu
folder_path = f'./faiss_{LANGCHAIN_PROJECT}'
if not os.path.exists(folder_path):
    print('Vector Store 생성 중')
    vectorstore = FAISS.from_documents(
        documents=split_documents,
        embedding=embeddings,
    )
    vectorstore.save_local(folder_path=folder_path)
    print('Vector Store 생성 및 로컬 저장 완료')
else:
    vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print('Vector Store 로컬에서 불러옴')


Vector Store 로컬에서 불러옴


In [13]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")

In [14]:
# LLM과 검색엔진을 활용한 RAG 구현 (기존 코드와 동일)
retriever = vectorstore.as_retriever(k=5)
chat = ChatUpstage(model='solar-1-mini-chat', temperature=0)

In [15]:
client = OpenAI()
llm_model = "gpt-3.5-turbo"

In [63]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
persona_qa = """
## Role: 사회/과학 지식 전문가

## Instructions
답변은 on 과 off 중에 선택한다. 
"on"인 경우 search api를 호출한다. 
"off"인 경우 search api를 호출하지 않고, 답변도 생성하지 않는다. 

1. 사용자가 대화를 통해 사회/과학 지식에 관한 주제로 질문하면 "on" 
2. 사회/과학 지식과 관련되지 않은 개인적인 감정, 기분 등의 메세지에 대해서는 "off"
"""

# RAG 구현에 필요한 질의 분석 및 검색 이외의 일반 질의 대응을 위한 LLM 프롬프트
persona_function_calling = """
## Role: 사회/과학 지식 전문가

## Instruction
답변은 on 과 off 중에 선택한다. 
"on"인 경우 search api를 호출한다. 
"off"인 경우 search api를 호출하지 않고, 답변도 생성하지 않는다. 

1. 사용자가 대화를 통해 사회/과학 지식에 관한 주제로 질문하면 "on": search api 호출 
   - 질문이 물리, 화학, 생물, 지구과학, 천문학, 컴퓨터공학, 소프트웨어, 국제정치 등 지식을 묻는 것이라면 search api를 호출할 수 있어야 한다. 
   - 특히, "알려줘", "설명해줘"로 끝나는 query는 search api를 호출한다. 
   - 아래는 search api를 호출하는 질문의 예시이다. 
   - 예시:
    "메탄올의 화학식은 무엇인가요?" = "on"
    "달의 공전 주기는 얼마인가요?" = "on"
    "파이썬에서 여러 값들의 평균을 구하는 방법은?" = "on"
    "이란의 무기 리베이트가 미국 정치에 미치는 영향은?" = "on"
    "각 나라별 공교육 현황은?" = "on"
    "대중교통은 그 가치가 얼마나 될까?" = "on"
    "각 나라의 문해율에 대해 알려줘" = "on"
    "화성에서 살 수 있을까?"= "on"
    "Edison"이 누구야?" = "on"


2. 사용자의 대화 메시지가 사회/과학 지식과 관련되지 않은 경우 "off": search api 호출하지 않고 답변 생성하지 않음 
   - 개인적인 감정이나 간단한 인사 등은 search api를 호출하지 않는다. 
   - 아래는 대답을 하지 않는 질문의 예시이다.
   - 예시: 
    "오늘은 기분이 아주 나빠" = "off" 
    "너는 모든 것을 알고 있구나" = "off" 
    "대답을 잘해줘서 너무 고마워!" = "off" 
    "요새 기분이 들쭉날쭉해" = "off" 
    "안녕 너 이름이 뭐야" = "off" 
    "너는 지금 무엇을 하고 있어?" = "off"  
    "우리 재미있는 이야기 하자." = "off" 
    """
# if search_mode == "on":
#     # search api를 호출하여 검색 
# else:
#     # search api를 호출하지 않고 대답하지 않음 


# # Function calling에 사용할 함수 정의
# tools = [
#     {
#         "type": "function",
#         "function": {
#             "name": "search",
#             "description": "Search relevant documents based on user message history",
#             "parameters": {
#                 "properties": {
#                     "message_history": {
#                         "type": "array",
#                         "items": {
#                             "type": "object",
#                             "properties": {
#                                 "role": {"type": "string"},
#                                 "content": {"type": "string"}
#                             },
#                             "required": ["role", "content"]
#                         },
#                         "description": "The entire history of user messages to form a comprehensive search query."
#                     }
#                 },
#                 "required": ["message_history"],
#                 "type": "object"
#             }
#         }
#     },
# ]

In [64]:
tools = [
    {
        "type": "function",
        "function": {"name": "search",
                     "description": "Search relevant documents based on user message history",
                     "parameters": {"type": "object",
                                    "properties": {"message_history": {
                                                        "type": "array", 
                                                        "items": {"type": "object",
                                                                  "properties": {"role": {"type": "string"},"content": {"type": "string"}},
                                                                  "required": ["role", "content"]},
                                                        "description": "The entire history of user messages to form a comprehensive search query."}},
                "required": ["message_history"]
            }
        }
    }
]


In [65]:
# Groundedness Check 함수 추가
def check_groundedness(context, response):
    url = "https://api.upstage.ai/v1/solar/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.environ.get('UPSTAGE_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "solar-1-mini-groundedness-check",
        "messages": [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ],
        "temperature": 0
    }
    
    try:
        api_response = requests.post(url, headers=headers, json=data)
        api_response.raise_for_status()  # Raises an HTTPError for bad responses
        result = api_response.json()
        return result['choices'][0]['message']['content']
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {e}")
        return "Error in groundedness check"

In [66]:
def format_docs(docs):
    global references
    references = docs
    return '\n\n'.join(doc.page_content for doc in docs)

In [68]:

def answer_question(messages):
    global search_mode, references
    response = {"topk": "", "answer": "", "references": "", "groundedness": ""}

    try:
        # 질의 분석 및 검색 이외의 질의 대응을 위한 LLM 활용
        msg = [{"role": "system", "content": persona_function_calling}] + messages
        result = client.chat.completions.create(
            model=llm_model,
            messages=msg,
            tools=tools,
            temperature=0,
            seed=1,
            timeout=10
        )

        # 검색이 필요한 경우 RAG 체인 활용
        if result.choices[0].message.tool_calls:
            rag_chain = (
                {'context': retriever | format_docs, 'question': RunnablePassthrough()}
                | prompt
                | chat
                | StrOutputParser()
            )
            history = '\n'.join([f"{message['role']}: {message['content']}" for message in messages]) + '\n'
            response["answer"] = rag_chain.invoke(history)
            ref_content = [reference.page_content for reference in references]
            topk = [reference.metadata['docid'] for reference in references]
            
        # 검색이 필요하지 않은 경우 답변을 생성하지 않음
        else:
            response["answer"] = None
            ref_content = []
            topk = []

    except Exception as e:
        traceback.print_exc()
        return response

    # Groundedness Check 수행
    context = '\n'.join(ref_content)
    groundedness = check_groundedness(context, response["answer"])

    response["topk"] = topk
    response["references"] = ref_content
    response["groundedness"] = groundedness
  
    return response

In [69]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as eval_lines, open(output_filename, 'w') as output_lines:
        idx = 0
        for eval_line in eval_lines:
            j = json.loads(eval_line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}')
            print(f'Groundedness: {response["groundedness"]}\n')

            output = {
                "eval_id": j["eval_id"], 
                "question": j["msg"],  # 질문 전체 저장
                # "standalone_query": response["standalone_query"],
                "topk": response["topk"], 
                "answer": response["answer"], 
                "references": response["references"],
                "groundedness": response["groundedness"]
            }
            output_lines.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [70]:
# 평가 실행
eval_rag('../data/eval.jsonl', '../output/EXP04_GC.csv')

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Answer: 나무의 분류에 대해 조사하기 위해서는 생물 분류학에서 중요한 기준인 유사한 특징을 찾아야 합니다. 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 새로운 방법은 생물체의 유전자나 단백질의 구성을 조사하여 재분류하는 데 사용됩니다. 과학자들은 새로 발견된 생물체를 분류하기 위해 철저한 조사를 진행하며, 생물체의 구조, DNA, 그리고 생활사를 상세히 분석합니다.
Groundedness: grounded

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Answer: 2017년 현재, 전세계의 공공 교육 지출은 세계 GDP의 약 4%를 차지하고 있습니다. 이는 국가의 교육 체계를 강화하고, 국민들의 교육 기회를 확대하는 데 도움을 줍니다. 많은 국가들이 교육에 많은 예산을 할당하고 있으며, 이는 국가의 교육 수준과 경제 경쟁력을 강화하고자 하는 노력의 일환입니다.
Groundedness: grounded

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
API 요청 오류: 400 Client Error: Bad Request for url: https://api.upstage.ai/v1/solar/chat/completions
Answer: None
Groundedness: Error in groundedness check

Test 3
Question: [{'role': 'user', 'content': '통학 버스의 가치에 대해 말해줘.'}]
A

KeyboardInterrupt: 